In [106]:
questions = []
with open("requetes_as_they_are.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        line = line.replace("\n", "")
        questions.append(line)

In [107]:
ponctuations = [
    ".", ",", "?"
    ]

articles_fr = [
    "le", "la", "les",        # Définis
    "un", "une", "des",       # Indéfinis
    "du", "de la", "de l’", "des",  # Partitifs
    "l’",                     # Élision
    "au", "aux", "à la", "à l’",  # Contractions avec "à"
    "du", "des", "de la", "de l’",  # Contractions avec "de"
    "de", "d'"
]


In [109]:
mots_cles_requete = [
        "rubriques", "articles", "article", "bulletins", "recherches"
]

mots_cles_articles = [
        "qui parlent de la ville", "mais qui ne parlent", "portant sur", "portent sur", "traitant", "évoquant", 
        "évoquent la ville","et qui évoquent", "évoquent", "possédant le mot", "et parlant", "parlant", "mentionnant", 
        " liés à", "dans le domaine", "et qui contiennent les mots", "contiennent les mots", "le contenu parle", 
        "à propos", "sur", "qui concernent", "parlent", "qui parle", "contenant les mots",  "portant",
    ]

mots_cles_rubrique = [
         "et", "parlant", "mentionnant", "mais", "qui" #ou fin de phrase
    ]

mots_cles_titre = [
        "est", "contient", "évoque", "traite"
    ]

mot_cles_images = [
        "avec des images", "contenant une image", "et qui ont des images"
    ]

In [110]:
def identifie_image(requete, mot_cles_images):
    for mot in mot_cles_images:
        if mot in requete:
            requete = requete.replace(mot, "")
            return requete, True
    return requete, None

In [111]:
#afficher post traitement des images
with open("requetes_as_they_are.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()  
    #print(list_of_strings)  

with open("post_image.txt", 'w', encoding='utf-8') as f2:
        with open("results_images.txt", 'w', encoding='utf-8') as f3:
            for requete_test in list_of_strings:
                #print(requete_test)
                requete_test = requete_test.lower()
                requete_test = requete_test.replace("?", "").replace(".", "").replace("’", "'")
                post_images, image_info = identifie_image(requete_test, mot_cles_images)
                f2.write(post_images)
                if image_info == True: image_info = "True"
                else: image_info = "False"
                f3.write(image_info + "\n")



In [112]:
def identifie_return(requete, mots_cles_requete):
    positions = {}
    for mot in mots_cles_requete:
        index = requete.find(mot)
        if index != -1:
            positions[mot] = index

    # Trie les mots par position d'apparition dans la requête
    mot_le_plus_tot = min(positions, key=positions.get)
    requete = requete[positions[mot_le_plus_tot] + len(mot_le_plus_tot):]
    return requete, mot_le_plus_tot


In [113]:
#afficher resultats post traitement return value
with open("post_image.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()  
    #print(list_of_strings)  

with open("post_return.txt", 'w', encoding='utf-8') as f2:
        with open("results_return.txt", 'w', encoding='utf-8') as f3:
            for requete_test in list_of_strings:
                post_return, result_return = identifie_return(requete_test, mots_cles_requete)
                f2.write(post_return)
                f3.write(result_return + "\n")



In [114]:
def identifie_rubrique(requete, mots_cles_rubrique):
    if "rubrique" not in requete:
        return requete, None, None  # "rubrique" absent

    # Position du mot "rubrique"
    start_index = requete.find("rubrique") + len("rubrique ")

    # Texte après "rubrique"
    sous_texte = requete[start_index:]

    # Trouver le mot-clé suivant
    min_index = len(sous_texte)  # valeur maximale par défaut
    for mot in mots_cles_rubrique:
        index = sous_texte.find(mot)
        if index != -1 and index < min_index:
            min_index = index

    resultat = sous_texte[:min_index]
    resultat = resultat.replace("est ", "").replace("'", "")
    if "ou" in resultat:
        resultat = resultat.split(" ou ")
        op_rubrique = "ou"
    else:
        op_rubrique = None

    requete_reste = requete[:start_index - len("rubrique ")] + sous_texte[min_index:]
    
    return requete_reste, resultat, op_rubrique

In [ ]:
requete = " qui parlent des systèmes embarqués dans la rubrique horizons enseignement"
requete_reste, resultat, op_rub = identifie_rubrique(requete, mots_cles_rubrique)
print("requete_reste:", requete_reste)
print("resultat:", resultat)
print("op_rub:", op_rub)

requete_reste:  qui parlent des systèmes embarqués dans la 
resultat: horizons enseignement
op_rub: None


In [115]:
#afficher resultats post traitement rubrique
with open("post_return.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()  
    #print(list_of_strings)  

with open("post_rubrique.txt", 'w', encoding='utf-8') as f2:
        with open("results_rubrique.txt", 'w', encoding='utf-8') as f3:
            for requete_test in list_of_strings:
                post_rubrique, result_rubrique, op_rubrique = identifie_rubrique(requete_test, mots_cles_rubrique)
                written_post = post_rubrique.replace("\n", "").replace("\n", "") + "\n"
                if written_post != "\n" and written_post != "":
                    f2.write(written_post)
                if result_rubrique is None:
                    result_rubrique = "None"
                elif isinstance(result_rubrique, list):
                    result_rubrique = "\t".join(result_rubrique)
                if op_rubrique is None:
                    op_rubrique = "op_None"
                f3.write(result_rubrique + "\t" + op_rubrique + "\n")



In [116]:
mots_cles_titre = [
        "est", "contient", "évoque", "traite"
    ]

def identifie_titres(requete, mots_cles_titre):
    if "titre" not in requete:
        return requete, None, None  # "titre" absent

    # cas très niche: contenant mot congres dans titre
    if "contenant" in requete and "dans titre" in requete:
        start_index = requete.find("contenant") + len("contenant")
        sous_texte = requete[start_index:]
        min_index = sous_texte.find("dans titre")
        resultat = sous_texte[:min_index]
        resultat = resultat.replace("mot ", "").replace("terme ", "")
        requete_reste = requete[: requete.find("contenant")]
        return requete_reste, resultat, None
    #les autres cas
    else:
        # Position du mot "titre"
        start_index = requete.find("titre") + len("titre ")

        # Texte après "titre"
        sous_texte = requete[start_index:]

        # Ou on arrete de regarder les titres
        min_index = len(sous_texte)  # valeur maximale par défaut
        if " ou " in sous_texte:
            min_index = sous_texte.find("ou")
        resultat = sous_texte[:min_index]
    #requete_reste = requete.replace(resultat, "")

    #sous_texte = partie qui contient le titre. maintenant, il faut le nettoyer et enlever les parties inutiles
    for mot in mots_cles_titre:
        resultat = resultat.replace(mot, "")
    resultat = resultat.replace("mot ", "").replace("terme ", "")

    #vérifier si il y a un "et" dans le titre
    op_titre = None
    if "et" in resultat:
        resultat = resultat.split(" et ")
        op_titre = "et"
    if isinstance(resultat, list):
        for res in resultat:
            res.replace("\n", "").replace("\t", "").replace("  ", " ")
    elif resultat in "    \t  \n":
        resultat = None
    else:
        resultat = resultat.replace("\n", "").replace("\t", "").replace("  ", " ")
 
    requete_reste = requete[:start_index - len("titre ")] + requete[min_index + len(sous_texte):]
    requete_reste = requete_reste.replace("dont", "").replace("dont", "")
    
    return requete_reste, resultat, op_titre


In [ ]:
requete = " dont le titre contient le mot chimie"
for article in articles_fr:
    requete = requete.replace(article + " ", "")
    
requete_result, resultat, op_titre = identifie_titres(requete, mots_cles_titre)
print("-----------------------------------")
print("requete_result:", requete_result)
print("resultat:", resultat)
print("op_titre:", op_titre)

-----------------------------------
requete_result:   
resultat:  chimie
op_titre: None


In [117]:
with open("post_rubrique.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()  
    #print(list_of_strings)  

with open("post_titre.txt", 'w', encoding='utf-8') as f2:
        with open("results_titre.txt", 'w', encoding='utf-8') as f3:
            for requete_test in list_of_strings:
                #print(requete_test)
                
                for article in articles_fr:
                    requete_test = requete_test.replace( " " + article + " ", " ")
                
                post_titres, titre, op_titre = identifie_titres(requete_test, mots_cles_titre)
                f2.write(post_titres)
                if titre == None: titre = "None"
                if op_titre == None: op_titre = "None"
                if isinstance(titre, list):
                    for i in titre:
                        f3.write(i + " ")
                    f3.write("\n")
                else:
                    f3.write(titre + "\n")


In [118]:
def premier_index_result(texte, mots_cles_dates):
    # Cherche l'index de chaque mot-clé dans le texte
    indices = []
    for mot in mots_cles_dates:
        idx = texte.find(mot)
        if idx != -1:
            indices.append(idx)
    # Cherche l'index du premier chiffre
    for i, c in enumerate(texte):
        if c.isdigit():
            indices.append(i)
            break
    if indices:
        ind = min(indices)
        return ind, texte[ind:]
    else:
        return None, texte
    
def last_index_result(texte, fin_dates):
    fin_dates = [
    "évoquant", "sur", "parlant", "portant", "parlent"
    ]
    indices = []
    for mot in fin_dates:
        idx = texte.find(mot)
        if idx != -1:
            indices.append(idx)
    if indices:
        ind = min(indices)
        print(ind, texte[:ind])
        return ind, texte[:ind]
    else:
        return None, texte

def contient_nombre(requete):
    return any(token.isdigit() or "/" in token for token in requete.split())





In [119]:
def clean_date(date_string):
    format_normalise_date = {"j": None, "m": None, "a": None}

    mois = ["janvier", "février", "mars", "avril", "mai", "juin", "juillet",
        "août", "septembre", "octobre", "novembre", "décembre"]
    date_string = date_string.replace("\n", "").replace("\t", "").replace("  ", " ")
    if "/" in date_string:
        date = date_string.split("/")
        format_normalise_date["j"] = date[0].replace(" ", "").replace("\n", "")
        format_normalise_date["m"] = date[1].replace(" ", "").replace("\n", "")
        format_normalise_date["a"] = date[2].replace(" ", "").replace("\n", "")

    else:
        date = date_string.split(" ")
        for date_info in date:
            if date_info.isdigit():
                if len(date_info) == 4:
                    format_normalise_date["a"] = date_info.replace(" ", "").replace("\n", "")
                else:
                    format_normalise_date["j"] = date_info.replace(" ", "").replace("\n", "")
            else:
                if date_info in mois:
                    mois_value = mois.index(date_info) + 1
                    if mois_value < 10:
                        format_normalise_date["m"] = "0" + str(mois_value)
                    else:
                        format_normalise_date["m"] = str(mois_value)

    return format_normalise_date

In [120]:

mots_cles_dates = [
"parus", "publiés" , "datés",
"à partir", "l'année", "écrits", "date", "datent", "mois"
]

mois = ["janvier", "février", "mars", "avril", "mai", "juin", "juillet",
        "août", "septembre", "octobre", "novembre", "décembre"]

fin_dates = [
    "évoquant", "sur", "parlant", "portant", "parlent"
    ]

def identifie_dates(requete, mots_cles_dates):
    resultat = {"début": None, "fin": None, "précis": None, "not": None}
    #vérifier si il y a une date dans la requete
    if not contient_nombre(requete):
        return requete, resultat
    
    #début et fin de partie date
    idx_debut, sous_texte = premier_index_result(requete, mots_cles_dates)
    idx_fin, sous_texte =  last_index_result(sous_texte, fin_dates)
    if idx_debut is None:
        idx_debut = 0
    if idx_fin is None:
        idx_fin = -1
    #clean up requete_reste
    #print(requete[idx_fin:])
    #print(requete[:idx_debut])
    #requete_reste = requete[:idx_debut] + requete[idx_fin:]
    requete_reste = requete.replace(sous_texte, "")

    #clean up sous_texte
    mots_trash = ["qui", "dont", "mais"]
    for mot in mots_trash:
        sous_texte = sous_texte.replace(mot, "")
    
    #3 cas possibles:
    #cas niche: on veut pas une date précise
    if "pas" in sous_texte:
        pas_indx = sous_texte.find("pas") + len("pas")
        not_date = sous_texte[pas_indx:]
        resultat["not"] = clean_date(not_date)
        sous_texte = sous_texte[:pas_indx - len("pas")]

    #cas 1: date entre deux dates
         #parus entre .... et .... / écrits entre .... et .... / publiés entre .... et ....
    if "entre" in sous_texte:
        sous_texte = sous_texte[sous_texte.find("entre") + len("entre"):]
        result_temp = sous_texte.split("et")
        #print(result_temp)
        resultat["début"] = clean_date(result_temp[0])
        resultat["fin"] = clean_date(result_temp[1])


    #cas 2: date à partir d'une date
        #datés à partir de .... / à partir ... / écrits après / date d'après ... / publiés après ...
    elif "à partir" in sous_texte or "après" in sous_texte:
        idx_à_partir = sous_texte.find("à partir")
        idx_apres = sous_texte.find("après")
        if idx_à_partir == -1:
            start_apres = idx_apres + len("après")
        else:
            start_apres = idx_à_partir + len("à partir")
            
        resultat["début"] = clean_date(sous_texte[start_apres:])

    #cas 3: date précise (utiliser else)
        #directement date / publiés mois ... /
    else:
        resultat["précis"] = clean_date(sous_texte)

    
    return requete_reste, resultat


In [ ]:
with open("post_titre.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()   

with open("post_dates.txt", 'w', encoding='utf-8') as f2:
    for requete_test in list_of_strings:                
        post_date, result_dates = identifie_dates(requete_test, mots_cles_dates)
        f2.write(post_date)
        print("--------------------")
        print(requete_test)
        for key, value in result_dates.items():
            print(key, value)
        print("reste:", post_date)




--------------------
 qui parlent systèmes embarqués dans 

début None
fin None
précis None
not None
reste:  qui parlent systèmes embarqués dans 

--------------------
 qui parlent cuisine moléculaire

début None
fin None
précis None
not None
reste:  qui parlent cuisine moléculaire

--------------------
 sur réalité virtuelle 

début None
fin None
précis None
not None
reste:  sur réalité virtuelle 

--------------------
 qui parlent d'airbus ou projet taxibot

début None
fin None
précis None
not None
reste:  qui parlent d'airbus ou projet taxibot

--------------------
 qui parlent tennis

début None
fin None
précis None
not None
reste:  qui parlent tennis

--------------------
 traitant lune

début None
fin None
précis None
not None
reste:  traitant lune

38 parus entre 3 mars 2013 et 4 mai 2013 
--------------------
 parus entre 3 mars 2013 et 4 mai 2013 évoquant etats-unis 

début {'j': '3', 'm': '03', 'a': '2013'}
fin {'j': '4', 'm': '05', 'a': '2013'}
précis None
not None
reste:  é

In [27]:
import spacy
nlp = spacy.load("fr_core_news_sm")
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
stopwords = list(fr_stop)

with open("post_dates.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()   

with open("post_clean_mots.txt", 'w', encoding='utf-8') as f2:
    for requete_test in list_of_strings:                
        for stop_word in stopwords:
            requete_test = requete_test.replace(" " + stop_word + " ", " ")
        f2.write(requete_test)



In [121]:
def contains_letter(s):
    return any(c.isalpha() for c in s)

In [122]:
mots_cles_articles = [
    "liés à", "domaine", "impliquant", "parlent",  "sur", "traitant", 
    "évoquant", "parlant", 
    "mentionnant", "portant", "portent",   "mots", "mot", "parle",
    "à propos", "évoquent",  "mentionnent"
]

niche = "cité"

trash_words = ["est-il", "qui ", "ville", "d'", "l'", "dans", "  ", "\n",
                "  ", ",", "mais", "ont été", "ont pour", "provenant", "dont"]
stop_word = ["et", "en", "soit"]

def clean_up(mot_cle):
    mot_cle = mot_cle.replace("\n", "").replace("\t", "")
    return mot_cle

def identifie_mots_cles(requete, mots_cles_articles):
    resultat = {"yes": [], "no": None}
    op_mots_cles = None

    if not contains_letter(requete):
        return resultat, None
    #enlever les mots inutiles
    for word in trash_words:
        requete = requete.replace(word, "")
    #requete = requete.replace(" en ", " ")
    
    #identifier les mots clés non voulues
    if "pas" in requete:
        pas_indx = requete.find("pas") + len("pas")
        not_mots_cles = requete[pas_indx:]
        resultat["no"] = not_mots_cles
        requete = requete[:pas_indx - len("pas")]
    #cas niche
    if niche in requete:
        resultat["yes"] = requete.replace(niche, "").replace("est-il", "").replace("qui", "")
        return resultat, None
    #sinon
    for mot in mots_cles_articles:
        if mot in requete:
            requete = requete[requete.find(mot) + len(mot):]

    if "ou " in requete or "soit" in requete:
        op_mots_cles = "ou"
        if "soit" in requete:
            sous_text = requete.split("soit")
        else:
            sous_text = requete.split("ou ")
        for i in range(len(sous_text)):
            sous_text[i] = clean_up(sous_text[i])
            if sous_text[i] != " " and sous_text[i] != "\n" and sous_text[i] != "" and sous_text[i] != "  ":
                resultat["yes"].append(sous_text[i])
    elif "et " in requete:
        sous_text = requete.split(" et")
        for i in range(len(sous_text)):
            sous_text[i] = clean_up(sous_text[i])
            if sous_text[i] != " " and sous_text[i] != "\n" and sous_text[i] != "" and sous_text[i] != "  ":
                resultat["yes"].append(sous_text[i])
    elif " en " in requete:
        sous_text = requete.split("en")
        for i in range(len(sous_text)):
            sous_text[i] = clean_up(sous_text[i])
            if sous_text[i] != " " and sous_text[i] != "\n" and sous_text[i] != "" and sous_text[i] != "  ":
                resultat["yes"].append(sous_text[i])
    else:
        resultat["yes"].append(requete.replace(" et", "").replace("»", "").replace("«", ""))

    return resultat, op_mots_cles

In [104]:
with open("post_dates.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()   

with open("post_mots.txt", 'w', encoding='utf-8') as f2:
    for requete in list_of_strings:  
        print("requete:", requete)
        print(identifie_mots_cles(requete, mots_cles_articles))    
        print("-------------------------------------------------")

requete:  qui parlent systèmes embarqués dans 

({'yes': [' systèmes embarqués'], 'no': None}, None)
-------------------------------------------------
requete:  qui parlent cuisine moléculaire

({'yes': [' cuisine moléculaire'], 'no': None}, None)
-------------------------------------------------
requete:  sur réalité virtuelle 

({'yes': [' réalité virtuelle '], 'no': None}, None)
-------------------------------------------------
requete:  qui parlent d'airbus ou projet taxibot

({'yes': [' airbus ', 'projet taxibot'], 'no': None}, 'ou')
-------------------------------------------------
requete:  qui parlent tennis

({'yes': [' tennis'], 'no': None}, None)
-------------------------------------------------
requete:  traitant lune

({'yes': [' lune'], 'no': None}, None)
-------------------------------------------------
requete:  évoquant etats-unis 

({'yes': ['ats-unis '], 'no': None}, None)
-------------------------------------------------
requete:  

({'yes': [], 'no': None}, None)
-

In [47]:
with open("post_dates.txt", 'r', encoding='utf-8') as f1:
    list_of_strings = f1.readlines()   

with open("post_mots.txt", 'w', encoding='utf-8') as f2:
    for requete in list_of_strings:                
        for mot in mots_cles_articles:
            if mot in requete:
                sous_text = requete[requete.find(mot) + len(mot):]
                if sous_text[0] != " ":
                    sous_text = sous_text.split(" ")[1:]
                    sous_text = " ".join(sous_text)
                    print("----------------------")
                    print("sous_text:", sous_text)
                else:
                    print("----------------------")
                    print("sous_text:", sous_text)




----------------------
sous_text:  systèmes embarqués dans 

----------------------
sous_text: systèmes embarqués dans 

----------------------
sous_text:  cuisine moléculaire

----------------------
sous_text: cuisine moléculaire

----------------------
sous_text:  réalité virtuelle 

----------------------
sous_text:  d'airbus ou projet taxibot

----------------------
sous_text: d'airbus ou projet taxibot

----------------------
sous_text:  tennis

----------------------
sous_text: tennis

----------------------
sous_text:  lune

----------------------
sous_text:  etats-unis 

----------------------
sous_text:  d'innovation

----------------------
sous_text:  russie ou japon 

----------------------
sous_text:  l'enseignement

----------------------
sous_text:  santé

----------------------
sous_text:  nutrition ou vins

----------------------
sous_text:  l'aéronautique

----------------------
sous_text:  serious game et réalité virtuelle

----------------------
sous_text:  d'informa

TODO:
    rajouter un moyen de nettoyer les résultats pour enlever les espaces inutiles et les \n

In [125]:
def traiter_requete_sans_re(requete):   
    # suppression des articles et ponctuations inutiless
    requete = requete.lower()
    requete = requete.replace("?", "").replace(".", "").replace("’", "'")
    # Structure pour stocker les composants de la requête
    resultats = {
        "return": None,
        "mots_cles": None,
        "rubrique": None,
        "dates": {"debut": None, "fin": None},
        "titre": None,
        "images": None
    }
    # Extraction des mots-clés
    requete, resultats["return"] = identifie_return(requete, mots_cles_requete)
    requete, resultats["images"] = identifie_image(requete, mot_cles_images)
    requete, resultats["rubrique"], op_rubrique = identifie_rubrique(requete, mots_cles_rubrique)
    if op_rubrique:
        resultats["operateurs_rubrique"] = op_rubrique
    
    #enlever les articles pour faciliter le traitement des mots clés
    for article in articles_fr:
        requete = requete.replace(" " + article + " ", " ")

    #Extraction des titres
    requete, resultats["titre"], op_titre = identifie_titres(requete, mots_cles_titre)
    if op_titre:
        resultats["operateurs_titre"] = op_titre
    
    # Extraction des dates (formats simples)
    requete, resultats["dates"] = identifie_dates(requete, mots_cles_dates)
    resultats["mots_cles"], op_mot_cle = identifie_mots_cles(requete, mots_cles_articles)
    if op_mot_cle:
        resultats["operateurs_mots_cles"] = op_mot_cle

    return resultats


# Exemple d'utilisation
if __name__ == "__main__":
    requete = input("Entrez votre requête en langage naturel : ")
    resultat = traiter_requete_sans_re(requete)
    print("Représentation structurée de la requête :")
    print(resultat)

Représentation structurée de la requête :
{'return': 'articles', 'mots_cles': {'yes': ['  '], 'no': None}, 'rubrique': 'focus ', 'dates': {'début': None, 'fin': None, 'précis': None, 'not': None}, 'titre': None, 'images': True}
